# Processing pipeline for Label-Free in vivo Quantification of the Vascular Network in Murine Colitis using Transrectal Absorber Guide Raster-Scanning Optoacoustic Mesoscopy (TAG-RSOM)

The following code provides a pipeline to pick up results generaded by the Fiji-pipeline and calculate the descriptors.

In [1]:
import os
import pandas as pd
import numpy as np

# Choose Working Directory

In [22]:
ANALYSIS_DIR = r"C:\\Path\\to\\Results"
RESULTS = r"C:\\Path\\to\\Results"
PIXELSIZE = 20; # in µm

# Normalized Number of Branches
The number of branches (vessels) in a skeleton is calculated by Fiji's analyse skeleton function and saved as file ending with "_BranchInfo.csv".

To normalize the number of branches it is devided by the analysis area of the used colon 2D image in cm². The colon area is saved as number of pixels in a file ending with "_SegmentedArea.csv".

(AREA_COLON_pixel * 20µm * 20µm) / 10^8 = AREA_COLON_cm²

    Normalized Number of Branches (NNB) has the unit (cm^-2)

In [3]:
df_NNB =  pd.DataFrame(columns=['Name', 'Value'])

for skeleton_name in sorted([file for file in os.listdir(ANALYSIS_DIR) if file.endswith("_Skeleton.csv")]): # Iterate throu individuals
    df_area = pd.read_csv(os.path.join(ANALYSIS_DIR, skeleton_name.replace("_Skeleton.csv", "_SegmentedArea.csv")))
    AREA_COLON_pixel = df_area.iloc[0]["Area"]
    AREA_COLON_CM2 = AREA_COLON_pixel * (PIXELSIZE/(10**8))
    df_branches = pd.read_csv(os.path.join(ANALYSIS_DIR, skeleton_name.replace("_Skeleton.csv", "_BranchInfo.csv")))
    
    df_NNB.loc[len(df_NNB)] = [skeleton_name.replace("_Skeleton.csv", ""), df_branches.shape[0] / AREA_COLON_CM2]
    
df_NNB.to_csv(os.path.join(RESULTS, "Normalized Number of Branches.csv"), header=False, index=False)

### Length of Largest Component
The largest component is calculated as the number of pixels representing junction-, end-, or slab-pixels. This information is calculated by Fiji's analyse skeleton function and saved as file ending with "_Skeleton.csv".

    Length of Largest Component (LC) has the unit (mm)
    
Calculated by number of pixels in LC * 20 µm / 10^3

In [4]:
df_LC_L =  pd.DataFrame(columns=['Name', 'Value'])

for skeleton_name in sorted([file for file in os.listdir(ANALYSIS_DIR) if file.endswith("_Skeleton.csv")]): # Iterate throu individuals
    df_skeleton = pd.read_csv(os.path.join(ANALYSIS_DIR, skeleton_name))
    
    tmp_LC = 0
    for index, row in df_skeleton.iterrows():
        if(row['# End-point voxels'] + row['# Junction voxels'] + row['# Slab voxels'] > tmp_LC):
            tmp_LC = row['# End-point voxels'] + row['# Junction voxels'] + row['# Slab voxels']
            
    df_LC_L.loc[len(df_LC_L)] = [skeleton_name.replace("_Skeleton.csv", ""), tmp_LC * (PIXELSIZE/(10**3))]

df_LC_L.to_csv(os.path.join(RESULTS, "Largest Components_Total Length.csv"), header=False, index=False)

# Normalized Network Length
The Normalized Network Length is calculated by deviding the total number of junction-, end-, or slab-pixels by the colon area. This information is calculated by Fiji's analyse skeleton function and saved as file ending with "_Skeleton.csv". The colon area is saved as number of pixels in a file ending with "\_SegmentedArea.csv".

    Normalized Network Length (NNL) has the unit (µm^-1)

In [5]:
df_NNL =  pd.DataFrame(columns=['Name', 'Value'])

for skeleton_name in sorted([file for file in os.listdir(ANALYSIS_DIR) if file.endswith("_Skeleton.csv")]): # Iterate throu individuals
    df_skeleton = pd.read_csv(os.path.join(ANALYSIS_DIR, skeleton_name))
    df_area = pd.read_csv(os.path.join(ANALYSIS_DIR, skeleton_name.replace("_Skeleton.csv", "_SegmentedArea.csv")))
    AREA_COLON_pixel = df_area.iloc[0]["Area"]
    
    number_pixels = 0
    for index, row in df_skeleton.iterrows():
        number_pixels += row['# End-point voxels'] + row['# Junction voxels'] + row['# Slab voxels']
    
    df_NNL.loc[len(df_NNL)] = [skeleton_name.replace("_Skeleton.csv", ""), number_pixels/(AREA_COLON_pixel*PIXELSIZE)]

df_NNL.to_csv(os.path.join(RESULTS, "Normalized Network Length.csv"), header=False, index=False)

# Normalized Vessel Area
The normalized vessel area is calculated by the percentage of non-zero pixels in the colon area. This information is saved as % in a file "_SegmentedArea.csv".

    Normalized Vessel Area (NVA) has the unit (%)

In [6]:
df_NVA =  pd.DataFrame(columns=['Name', 'Value'])

for skeleton_name in sorted([file for file in os.listdir(ANALYSIS_DIR) if file.endswith("_Skeleton.csv")]):
    # Open nessesary files
    df_area = pd.read_csv(os.path.join(ANALYSIS_DIR, skeleton_name.replace("_Skeleton.csv", "_SegmentedArea.csv")))
    
    df_NVA.loc[len(df_NVA)] = [skeleton_name.replace("_Skeleton.csv", ""), df_area.iloc[0]["%Area"]]

df_NVA.to_csv(os.path.join(RESULTS, "Normalized Vessel Area.csv"), header=False, index=False)

# Average Vessel Diameter
The average vessel diameter is calculated by deviding the total vessel area by the total vessel length.

    Average Vessel Diameter (AVD) has the unit (µm)

In [7]:
df_AVD =  pd.DataFrame(columns=['Name', 'Value'])

for skeleton_name in sorted([file for file in os.listdir(ANALYSIS_DIR) if file.endswith("_Skeleton.csv")]): # Iterate throu individuals
    df_skeleton = pd.read_csv(os.path.join(ANALYSIS_DIR, skeleton_name))
    df_area = pd.read_csv(os.path.join(ANALYSIS_DIR, skeleton_name.replace("_Skeleton.csv", "_SegmentedArea.csv")))
    AREA_VESSEL_pixels = (df_area.iloc[0]["Area"] / 100) * df_area.iloc[0]["%Area"]
    
    number_pixels = 0
    for index, row in df_skeleton.iterrows():
        number_pixels += row['# End-point voxels'] + row['# Junction voxels'] + row['# Slab voxels']
    
    df_AVD.loc[len(df_AVD)] = [skeleton_name.replace("_Skeleton.csv", ""), (AREA_VESSEL_pixels*PIXELSIZE)/number_pixels]

df_AVD.to_csv(os.path.join(RESULTS, "Average Vessel Diameter.csv"), header=False, index=False)

# Normalized Blood Volume
The normalized Blood Volume (µm^3) is calculated based on the geographic distance map relative to the colon area (µm^2) imaged.

    Normalized Blood Volume (Distance Map) (NBV_DM) has the unit (µm)

In [8]:
df_NBV_DM =  pd.DataFrame(columns=['Name', 'Value'])


for skeleton_name in sorted([file for file in os.listdir(ANALYSIS_DIR) if file.endswith("_Skeleton.csv")]): # Iterate throu individuals
    # Open nessesary files
    df_area = pd.read_csv(os.path.join(ANALYSIS_DIR, skeleton_name.replace("_Skeleton.csv", "_SegmentedArea.csv")))
    AREA_COLON_pixel = df_area.iloc[0]["Area"] # Returns number of pixels
    AVD_DM = pd.read_csv(os.path.join(ANALYSIS_DIR, skeleton_name.replace("_Skeleton.csv", "_DistanceMap.csv")))
    # Drop first and zero lines
    AVD_DM = AVD_DM.drop(0)
    AVD_DM = AVD_DM[AVD_DM['Count'] != 0]
    AVD_DM_mm = ((AVD_DM['Distance']*PIXELSIZE)**2 * np.pi *AVD_DM['Count']*PIXELSIZE).sum() / (AREA_COLON_pixel*PIXELSIZE**2)
    
    df_NBV_DM.loc[len(df_NBV_DM)] = [skeleton_name.replace("_Skeleton.csv", ""), AVD_DM_mm]


df_NBV_DM.to_csv(os.path.join(RESULTS, "Normalized Blood Volume.csv"), header=False, index=False)

# Fractal Dimension
The fractal dimension is calculated by Fiji's box-counting method.

    Fractal Dimension (FD) has no unit

In [9]:
df_FD =  pd.DataFrame(columns=['Name', 'Value'])

for skeleton_name in sorted([file for file in os.listdir(ANALYSIS_DIR) if file.endswith("_Skeleton.csv")]):
    df_fd = pd.read_csv(os.path.join(ANALYSIS_DIR, skeleton_name.replace("_Skeleton.csv", "_BoxCount.csv")))
    df_FD.loc[len(df_FD)] = [skeleton_name.replace("_Skeleton.csv", ""), df_fd["D"].mean()]

df_FD.to_csv(os.path.join(RESULTS, "Fractal Dimension.csv"), header=False, index=False)